In [1]:
def gini_index(groups, classes):
    # total number of samples
    n_instances = sum(len(group) for group in groups)

    # weighted gini index for all groups
    gini = 0.0
    for group in groups:
        group_size = len(group)
        if size == 0:
            continue
        # calculate the score for the group
        score = 0.0
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p ** 2
        # weight by the size of the group
        gini += (1 - score) * (group_size / n_instances)

    return gini

In [2]:
def test_split(index, value, dataset):
    """Split the dataset based on an attribute and attribute value."""
    left, right = [], []
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

def get_best_split(dataset):
    """Find the best split point for a dataset."""
    class_values = list(set(row[-1] for row in dataset))
    best_index, best_value, best_score, best_groups = None, None, float('inf'), None
    for index in range(len(dataset[0]) - 1): # Exclude the header label
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < best_score:
                best_index, best_value, best_score, best_groups = index, row[index], gini, groups
    return {'index': best_index,
            'value': best_value,
            'groups': best_groups,
           }        

In [3]:
# build the tree
def to_terminal(group):
    """Create a terminal node value."""
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

def split(node, max_depth, min_size, depth):
    """recursive function to split a node."""
    left, right = node['groups']
    del node['groups']

    # check for no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return

    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return

    # Process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_best_split(left)
        split(node['left'], max_depth, min_size, depth + 1)

    # Process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_best_split(right)
        split(node['right'], max_depth, min_size, depth + 1)
    

In [4]:
# Create the Tree
def build_tree(train, max_depth, min_size):
    """Build a decision tree."""
    root = get_best_split(train)
    split(root, max_depth, min_size, 1)
    return root

In [5]:
# Make predictions:
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

In [6]:
# Example dataset
dataset = [
    [2.771244718, 1.784783929, 0],
    [1.728571309, 1.169761413, 0],
    [3.678319846, 2.81281357, 0],
    [3.961043357, 2.61995032, 0],
    [2.999208922, 2.209014212, 0],
    [7.497545867, 3.162953546, 1],
    [9.00220326, 3.339047188, 1],
    [7.444542326, 0.476683375, 1],
    [10.12493903, 3.234550982, 1],
    [6.642287351, 3.319983761, 1]
]

# Build the tree
tree = build_tree(dataset, max_depth=3, min_size=1)

# Make predictions
for row in dataset:
    prediction = predict(tree, row)
    print(f"Expected={row[-1]}, Predicted={prediction}")


Expected=0, Predicted=0
Expected=0, Predicted=0
Expected=0, Predicted=0
Expected=0, Predicted=0
Expected=0, Predicted=0
Expected=1, Predicted=1
Expected=1, Predicted=1
Expected=1, Predicted=1
Expected=1, Predicted=1
Expected=1, Predicted=1
